In [46]:
import numpy as np

In [ ]:
#EAE135 Project 3
#Andrew Matel
#919512498

#Given Info
E = 7.2395*(10**7) #Young's Modulus of Al 2024-T861 10.5 ksi to Pa
poisson = 0.33
R = 1.95 #Reference Length (m)
A_s = 150*(10**(-6))  #Stringer Area (m^2)
t = 2.95*(10**(-3)) #Skin Thickness (m)
c = 8*R #Chord Length (m)
V2 = 0 # Drag Force (N)
V3 = 256000 # Lift Force (N)
ultimateshear = 295 # (MPA)

x3 = np.zeros(6)
x2 = np.zeros(6) 

##Coords of each stringer from reference point
#Coords of Stringer 1
x3[0] = 0
x2[0] = 7*R

#Coords of Stringer 2
x3[1] = (6/7)*R
x2[1] = R

#Coords of Stringer 3
x3[2] = R
x2[2] = 0

#Coords of Stringer 4
x3[3] = 0
x2[3] = -R   

#Coords of Stringer 5
x3[4] = -R/3 
x2[4] = -(2/3)*R

#Coords of Stringer 6
x3[5] = -(6/23)*R
x2[5] = R


##MWC
top2 = 0
top3 = 0
bot = 0
for i in range(0, len(x3)):
    top2 = top2 + E*A_s*(x2[i])
    top3 = top3 + E*A_s*(x3[i])
    bot = bot +E*A_s
x2c = top2/bot
x3c = top3/bot

print("x2c = {} m\n".format(x2c))
print("x3c = {} m\n".format(x3c))

##Bending Stiffnesses
#Coords with respect to modulus weighted centroid
x2toc = [x2[0]-x2c, x2[1]-x2c, x2[2]-x2c, x2[3]-x2c, x2[4]-x2c, x2[5]-x2c]
x3toc = [x3[0]-x3c, x3[1]-x3c, x3[2]-x3c, x3[3]-x3c, x3[4]-x3c, x3[5]-x3c]

H22c = 0
H23c = 0
H33c = 0
for i in range(0, len(x3toc)):
    H22c = H22c + E*A_s*(x3toc[i])**2
    H23c = H23c + E*A_s*(x3toc[i])*(x2toc[i])
    H33c = H33c + E*A_s*(x2toc[i])**2

print("H22c = {} Nm^2\n".format(H22c))
print("H33c = {} Nm^2\n".format(H33c))
print("H23c = {} Nm^2\n".format(H23c))

##Open Shear Flow
delfo = np.zeros(5)
delH = H22c*H33c - (H23c)**2
fo = np.zeros(6)

for i in range(1, len(fo)):
    delfo[i-1] = -E*A_s*(((H22c*V2 - H23c*V3)/(delH))*x2toc[i] - ((H23c*V2-H33c*V3)/(delH))*x3toc[i])

for i in range(1, len(fo)):
    fo[i] = fo[i-1] + delfo[i-1]

print('list of open shear flow change due to each node in N/m: {}\n'.format(delfo))
print('list of open shear flow for each segment in N/m: {}\n'.format(fo))

##Closing Shear Flow
G = E/(2*(1+poisson))
l12 = (R**2 + (6*R)**2)**(0.5)
l23 = (((1/7)*R)**2 + R**2)**(0.5)
l34 = 0.25*(2*R*np.pi)
l45 = 0.25*(2*(R/3)*np.pi)
l56 = c
l61 = (((6/23)*R)**2 + (6*R)**2)**(0.5)
seglengths = [l12, l23, l34, l45, l56, l61]
perimeter = sum(seglengths)

print('Length of Each Section = {} m\n'.format(seglengths))

fctop = 0
for i in range(0, len(seglengths)):
    fctop = fctop + fo[i]*seglengths[i]
fc = -(fctop)/(perimeter)

print('Closing Shear Flow = {} N/m\n'.format(fc))

##Shear Center
A12 = 0.5*(x2[1]*x3[1])+0.5*(x3[1]*(x2[0]-R))
A23 = 0.5*(((1/7)*R)*R)+0.5*(((6/7)*R)*R)
A34 = 0.25*(np.pi)*(R**2)
A45 = 0.25*(np.pi*(R/3)**2)+0.5*(R/3)*((2/3)*R)

#Heron's formula
# a = ((2/3)*R**2 + (R/3)**2)**(0.5)
# b = ((R**2) + ((6/23)*R)**2)**(0.5)
# c = (((5/3)*R)**2 + (R/23)**2)**(0.5)
# s = (a+b+c)/2
# A56 = (s*(s-a)*(s-b)*(s-c))**(0.5)

def triangle_area(x1, y1, x2, y2, x3, y3):
    """
    Calculate the area of a triangle given its three vertices using the determinant formula.
    """
    area = 0.5 * abs(x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    return area

A56 = triangle_area(0, 0, -2*R/3, -R/3, R, -6*R/23)
A61 = triangle_area(0, 0, 7*R, 0, R, -6*R/23)
# A61 = 0.5*((6/23*R)*(6*R))
# A56 = ((R/3)*((8*R)-(R/3)))/2 - A61

# A61 = 0.5*(((6/23)*R)*R) + 0.5*(((6/23)*R)*(6*R))

Asections = [A12, A23, A34, A45, A56, A61]
print('Area of Each Section = {} m^2\n'.format(Asections))
Atot = sum(Asections)

x2k = 0
for i in range(0, len(fo)):
    x2k = x2k + (fo[i]*2*Asections[i])

x2k = (x2k + 2*fc*Atot)/(V3)
print('x2 Coordinate of Shear Center = {} m from the origin\n'.format(x2k))

##Shear Flow Due to Torsion
delx = R - x2k
M1 = V3*delx
ft = (M1)/(2*Atot)

print('Shear Flow Due to Torsion = {} N/m\n'.format(ft))

##Total Shear Flow of Each Segment
ftot = np.zeros(6)
for i in range(0, len(ftot)): 
    ftot[i] = fo[i] + fc + ft

print('Total Shear Flows in Each Section in N/m are: {}\n'.format(ftot))

x2c = 2.3833333333333333 m

x3c = 0.410455486542443 m

H22c = 68050.60316914854 Nm^2

H33c = 1795450.2962499994 Nm^2

H23c = -29940.953229813662 Nm^2

list of open shear flow change due to each node in N/m: [-51595.98164891 -61721.914884    19865.51010774  46169.1607428
  38123.60792079]

list of open shear flow for each segment in N/m: [      0.          -51595.98164891 -113317.89653291  -93452.38642517
  -47283.22568237   -9159.61776158]

Length of Each Section = [11.861386934081528, 1.9697974618768108, 3.0630528372500483, 1.0210176124166828, 15.6, 11.711053379886057] m^2

Closing Shear Flow = 30713.007944579324 N/m

Area of Each Section = [11.407499999999999, 1.9012499999999997, 2.9864765163187967, 0.7543307240354218, 0.9644021739130435, 3.471847826086956] m^2

x2 Coordinate of Shear Center = 0.5896939103012501 m from the origin

Shear Flow Due to Torsion = 8103.916112326129 N/m

Total Shear Flows in Each Section in N/m are: [ 38816.92405691 -12779.05759201 -74500.972476   -54635.46

In [48]:
##Safety Factor
Shear = np.zeros(6)
for i in range(0, len(Shear)):
    Shear[i] = (ftot[i]/t)*(10**(-6))

print("Shear Stress in each Section in MPa: {}\n".format(Shear))

SF = np.zeros(6)
for i in range(0, len(SF)):
    SF[i] = abs(ultimateshear/Shear[i])

print('Safety Factor for Each Segment: {}\n'.format(SF))

Shear Stress in each Section in MPa: [ 13.15827934  -4.33188393 -25.25456694 -18.52049572  -2.86993275
  10.05332417]

Safety Factor for Each Segment: [ 22.41934468  68.09970092  11.68105558  15.92829935 102.78986487
  29.34352808]

